# Python with PostgreSQL & PostGIS

<span>Note: Please always run the complete Jupyter Notebook from the beginning, as object names such as 'sql' and 'gdf' are reused in the code cells.</span>

## Libraries and Settings

In [1]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/python_postgresql_postgis


## Create database connection

In [2]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Current database:", db_name)
except Exception as e:
    print("Error connecting to the database:", e)

# Dispose the engine
engine.dispose()

Current database: osm_switzerland


## List tables in database

In [3]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public';"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

geography_columns
geometry_columns
spatial_ref_sys
planet_osm_polygon
planet_osm_point
planet_osm_line
planet_osm_roads


## Show columns and data types of a selected table

In [4]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Dispose the engine
engine.dispose()

# Print the DataFrame
df

column_name     data_type
0             osm_id        bigint
1            z_order       integer
2           way_area          real
3                way  USER-DEFINED
4   addr:housenumber          text
..               ...           ...
68              wood          text
69         tracktype          text
70            access          text
71    addr:housename          text
72       addr:street          text

[73 rows x 2 columns]

## Query: Select buildings for which the full address is available in defined zip code areas

In [5]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query 
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p."addr:street" IS NOT NULL
                AND p."addr:housenumber" IS NOT NULL
                AND p."addr:city" IS NOT NULL
                AND p."addr:postcode" IN ('8001', '8002')"""

# Query the database and store the result in a GeoDataFrame
gdf_buildings = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_buildings


osm_id          addr:street addr:housenumber addr:city addr:postcode  \
0     157954139         Löwenstrasse               67    Zürich          8001   
1     157954140         Löwenstrasse               69    Zürich          8001   
2     316823317         Bahnhofplatz               12    Zürich          8001   
3     157954132         Gessnerallee               54    Zürich          8001   
4     157954131         Gessnerallee               52    Zürich          8001   
...         ...                  ...              ...       ...           ...   
2348  108633096  Brandschenkestrasse             152c    Zürich          8002   
2349  108633055  Brandschenkestrasse             152b    Zürich          8002   
2350  108626600  Brandschenkestrasse             110a    Zürich          8002   
2351   34572240  Brandschenkestrasse              110    Zürich          8002   
2352  107080965         Gessnerallee               17    Zürich          8001   

        building                                               geom  
0     commercial  POLYGON ((8.53805 47.37717, 8.53808 47.37716, ...  
1     commercial  POLYGON ((8.53813 47.37725, 8.5383 47.37718, 8...  
2     commercial  POLYGON ((8.53819 47.37733, 8.5382 47.37732, 8...  
3     apartments  POLYGON ((8.53781 47.37744, 8.53797 47.37736, ...  
4     apartments  POLYGON ((8.53775 47.37738, 8.53791 47.3773, 8...  
...          ...                                                ...  
2348      office  POLYGON ((8.52388 47.36409, 8.52391 47.3639, 8...  
2349  industrial  POLYGON ((8.5239 47.36435, 8.5239 47.36431, 8....  
2350      office  POLYGON ((8.52458 47.3655, 8.52462 47.36534, 8...  
2351  commercial  POLYGON ((8.52469 47.36579, 8.52476 47.36552, ...  
2352     service  POLYGON ((8.53561 47.37627, 8.53566 47.37624, ...  

[2353 rows x 7 columns]

## Show selected features on map

<span">Note the popup field in the map, which has been added to provide additional information about buildings.</span>

<span">Example of alternative background maps (maptiles) are:</span>
- <span">EsriWorldImagery</span>
- <span">EsriWorldTopoMap</span>
- <span">EsriWorldGrayCanvas</span>
- <span">CartoDBDarkMatter</span>
- <span">CartoDBPositron</span>


In [6]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_buildings.crs is None:
    gdf_buildings.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_buildings.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15,
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_buildings,
    name='geojson',
    weight=0.5,
    fill_color='greenyellow',
    fillOpacity=0.8,
    popup=folium.GeoJsonPopup(fields=['addr:street',
                                      'addr:housenumber',
                                      'addr:city',
                                      'addr:postcode',
                                      'building'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_buildings.html')


## Query: Select coffee stores

In [7]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            h.osm_id,
            h.shop,
            h.name,
            ST_Transform(h.way, 4326) AS geom
        FROM planet_osm_point h
        WHERE h.shop = 'coffee';"""

# Query the database and store the result in a GeoDataFrame
gdf_coffee_stores = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_coffee_stores.head(10)


osm_id    shop                          name                      geom
0    708997948  coffee            Nespresso Boutique   POINT (8.72977 47.4982)
1   4153947943  coffee  Royal Pacific Coffee Company  POINT (8.72245 47.50071)
2   4447099306  coffee        Rösterei Caffè l’amica  POINT (8.72165 47.50352)
3   7920930785  coffee                      Café etc  POINT (8.62947 47.39642)
4   4841545900  coffee            Nespresso Boutique  POINT (8.59574 47.40825)
5   2340186699  coffee         Kaffeerösterei Vitali  POINT (8.53944 47.42036)
6    311859455  coffee                        Tchibo  POINT (8.54651 47.40951)
7  12023129318  coffee                      Atinkana  POINT (8.59894 47.40057)
8   3830372125  coffee                        Tchibo  POINT (8.55472 47.36508)
9   5628118179  coffee                        Tchibo  POINT (8.52506 47.31128)

## Show selected features on map

In [8]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_coffee_stores.crs is None:
    gdf_coffee_stores.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_coffee_stores.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf_coffee_stores,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'shop'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_coffee_stores.html')


## Query: Select all supermarkets in a distance of 1000m around the central station in the city of Winterthur.

<span>Note:</span>

<span>For each supermarket, the distance to the central station in meters is calculated and stored as new column 'distance_meters'.</span>

<span>In addition, a popup field was added to the map, allowing users to view detailed information about each selected feature when they click on it.</span>

<span>The WGS84 (World Geodetic System 1984) coordinates in ST_MakePoint(LON, LAT) were derived from: https://tools.retorte.ch/map.</span>


In [9]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            p.osm_id,
            p.shop,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'supermarket'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography,
                1000
            )
        ORDER BY distance_meters;"""

# Query the database and store the result in a GeoDataFrame
gdf_supermarkets = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_supermarkets.head(10)


osm_id         shop                          name  distance_meters  \
0   706203439  supermarket                          Coop       159.883419   
1  4109460421  supermarket                     Asia Shop       162.391281   
2  3831772784  supermarket                        Migros       247.578208   
3  7380954145  supermarket                      Alnatura       256.838011   
4  4095400190  supermarket                          ALDI       274.275393   
5  4125136758  supermarket  Tandoor Indischer Supermarkt       290.212664   
6  4095400136  supermarket                        Denner       316.354037   
7   709022324  supermarket               Claro Weltladen       441.129317   
8   359694887  supermarket                        Denner       584.131181   
9  4058248551  supermarket                        Migros       600.117307   

                       geom  
0  POINT (8.72594 47.50085)  
1  POINT (8.72208 47.50101)  
2  POINT (8.72115 47.49916)  
3  POINT (8.72074 47.49958)  
4   POINT (8.72476 47.4979)  
5  POINT (8.72017 47.50073)  
6  POINT (8.72036 47.49886)  
7  POINT (8.72912 47.49842)  
8  POINT (8.71914 47.50442)  
9  POINT (8.73193 47.50012)

## Show selected features on map

In [10]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_supermarkets.crs is None:
    gdf_supermarkets.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_supermarkets.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf_supermarkets,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'distance_meters'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_supermarkets.html')


## Query: Select all roads classified as 'motorway' and create a 5000m buffer around these roads.

In [11]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query (major roads)
sql = """-- Create buffer around major roads
        SELECT 
            1 as group_id,
            ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 2000)), 4326) AS geom
        FROM public.planet_osm_roads AS p
        WHERE
            highway = 'motorway';"""

# Query the database and store the result in a GeoDataFrame
gdf_roads = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_roads

group_id                                               geom
0         1  MULTIPOLYGON (((8.35551 47.41963, 8.3551 47.41...

## Show selected features on map

In [12]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_roads.crs is None:
    gdf_roads.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_roads.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf_roads,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_roads.html')


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [13]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-10-05 15:36:43
Python Version: 3.12.1
-----------------------------------
